In [1]:
# Import libraries
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import sklearn.model_selection as sms
import sklearn.linear_model as slm
import sklearn.preprocessing as skp
import sklearn.feature_selection as skf
from sklearn.cluster import DBSCAN
import numpy as np
from IPython.display import HTML
import util
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sklearn import linear_model
from sklearn import datasets
from scipy.optimize import minimize
%matplotlib inline




In [2]:
def soft_threshold(rho,lamda):
    '''Soft threshold function used for normalized data and lasso regression'''
    if rho < - lamda:
        return (rho + lamda)
    elif rho >  lamda:
        return (rho - lamda)
    else: 
        return 0
    
def soft_thresholdv(rho,lamda):
    '''Array version of soft threshold function'''
    s = np.zeros_like(rho)
    s[rho < -lamda] = rho[rho < -lamda]+lamda
    s[rho > lamda] = rho[rho > lamda]-lamda
    return s
    

def coordinate_descent_lasso(theta,X,y,lamda,num_iters,intercept):
    '''Coordinate gradient descent for lasso regression - for normalized data. 
    The intercept parameter allows to specify whether or not to regularize theta_0'''
    # Helpful dimensions 
    m,n = X.shape
    # Normalizing X in case it was not done before
    X = X / (np.linalg.norm(X,axis = 0))
    # Looping until max number of iterations
    for i in range(num_iters): 
        # Looping through each coordinate
        for j in range(n):
            # Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y-y_pred+theta[j]*X_j)
            # Checking intercept parameter
            if intercept == True:  
                if j == 0: 
                    theta[j] = rho 
                else:
                    theta[j] = soft_threshold(rho, lamda)  
            if intercept == False:
                theta[j] = soft_threshold(rho, lamda)   
    return theta.flatten()

def gradient_descent_reg(theta,X,y,alpha,lamda,num_iters):
    '''Gradient descent for lasso regression'''
    # Helpful dimensions 
    m = np.size(y)
    for i in range(num_iters):
        # Prediction
        h = np.dot(X,theta)
        rho = np.dot(X.T,(h-y))
        # Gradient function in vectorized form
        theta = theta-alpha*(1/m)*(soft_thresholdv(rho,lamda))
    return theta.flatten()

def conjugate_gradient_descent(X, y, theta0, lamda, num_iters):
    theta = theta0
    m = len(y)
    for k in np.arange(num_iters):
        F = (1/m)*(X*theta-y)**2+lamda*np.sum(np.abs(theta))
        h = np.dot(X,theta)
        rho = np.dot(X.T,(h-y))
        p = -soft_thresholdv(rho,lamda)    
        alpha = 1e-4
        theta = theta+alpha*p
    return np.array(theta)

In [3]:
# Get case IDs
case_list = open('/home/ali/RadDBS-QSM/data/docs/cases_90','r')
lines = case_list.read()
lists = np.loadtxt(case_list.name,comments="#", delimiter=",",unpack=False,dtype=str)
case_id = []
for lines in lists:     
    case_id.append(lines[-9:-7])

# Load scores
file_dir = '/home/ali/RadDBS-QSM/data/docs/QSM anonymus- 6.22.2023-1528.csv'
motor_df = util.filter_data(file_dir,'pre-dbs updrs','stim')
# Find cases with all required scores
subs,pre_imp,post_imp,pre_updrs_off = util.get_full_cases(motor_df,
                                                          'CORNELL ID',
                                                          'OFF (pre-dbs updrs)',
                                                          'ON (pre-dbs updrs)',
                                                          'OFF meds ON stim 6mo')
# Load extracted features
npy_dir = '/home/ali/RadDBS-QSM/data/npy/'
phi_dir = '/home/ali/RadDBS-QSM/data/phi/phi/'
roi_path = '/data/Ali/atlas/mcgill_pd_atlas/PD25-subcortical-labels.csv'
n_rois = 6
all_rois = False
Phi_all, X_all, R_all, K_all, ID_all = util.load_featstruct(phi_dir,npy_dir+'X/',npy_dir+'R/',npy_dir+'K/',n_rois,1595,all_rois)
ids = np.asarray(ID_all).astype(int)
# Find overlap between scored subjects and feature extraction cases
c_cases = np.intersect1d(np.asarray(case_id).astype(int),np.asarray(subs).astype(int))
# Complete case indices with respect to feature matrix
c_cases_idx = np.in1d(ids,c_cases)

X_all_c, K, R, subsc, pre_imp, pre_updrs_off, per_change = util.re_index(X_all,K_all,R_all,c_cases_idx,subs,ids,all_rois,pre_imp,pre_updrs_off,post_imp)

4
('CORNELL ID', 'OFF (pre-dbs updrs)', 'ON (pre-dbs updrs)', 'OFF meds ON stim 6mo')


NameError: name 'h0' is not defined

In [ ]:
K0 = K.reshape(1,-1)
results_ls = np.zeros_like(per_change)
cvn = 5
alphas = np.logspace(-4,-1,10)
gd_cd = True

In [ ]:
for j in np.arange(len(subsc)):
      test_id = subsc[j]
      test_index = subsc == test_id
      train_index = subsc != test_id
      X_train = X_all_c[train_index,:,:]
      X_test = X_all_c[test_index,:,:]
      y_train = per_change[train_index]
      y_test = per_change[test_index]
      # Cross validation
      X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                  X_train,train_index,X_test,
                                                  test_index,pre_updrs_off,all_rois,False,False)
      with np.errstate(divide='ignore', invalid='ignore'):
        # Feature selection, make sure A is square
        sel = skf.SelectKBest(skf.r_regression,k=X0_ss0.shape[0])
        X0_ss = (sel.fit_transform(X0_ss0,y_train))
        X_test_ss = (sel.transform(X_test_ss0))
        K = sel.transform(K0)
        y_n = cKDTree(X0_ss).query(X_test_ss, k=1)[1]

      # Initialize variables
      X = X0_ss
      m,n = X.shape
      initial_theta = np.zeros((n,1))
      bnds = []
      for j in np.arange(len(initial_theta)): 
        bnds.append((-1e1, 1e1))
      theta_list_cd = []
      theta_list_gd = []
      theta_list_cg = []
      lamda = np.flip(np.logspace(-4,1,100)) # Range of lambda values
      y = y_train.reshape(-1,1)

      
      #Run lasso regression for each lambda
      if gd_cd == True:
        for l in lamda:
            theta_cd = coordinate_descent_lasso(initial_theta,X,y,lamda = l,num_iters=100,intercept=False)
            theta_list_cd.append(theta_cd)
            theta_gd = gradient_descent_reg(initial_theta,X,y,alpha=1e-3,lamda = l,num_iters=100)
            theta_list_gd.append(theta_gd)
            theta_cg = conjugate_gradient_descent(X,y,np.ones((n,1)),l,num_iters=100)
            theta_list_cg.append(theta_cg)
      #Stack into numpy array
      theta_lasso_cd = np.stack(theta_list_cd).T
      theta_lasso_gd = np.squeeze(np.stack(theta_list_gd).T)
      theta_lasso_cg = np.stack(theta_list_cg).T
      try:
         np.linalg.cholesky(X0_ss)
      except:
         print('Matrix is not positive definite')
      kappa = np.linalg.cond(X0_ss)
      print('Condition number',str(kappa))
      print('Mean weights:',str(np.mean(theta_lasso_cd)),str(np.mean(theta_lasso_gd)),str(np.mean(theta_lasso_cg)))
      print('CG-CD error',np.mean((theta_lasso_cd-theta_lasso_cg)**2))
      print('CG-GD error',np.mean((theta_lasso_gd-theta_lasso_cg)**2))
      print('GD-CD error',np.mean((theta_lasso_cd-theta_lasso_gd)**2))
      if np.mean((theta_lasso_cd-theta_lasso_cg)**2) < 1e-6:
        #Plot results
        n,_ = theta_lasso_cd.shape
        fig,ax = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(10,5))

        for i in range(10):
            ax[0].plot(lamda, theta_lasso_cd[i])#,label = K[:,i])

        ax[0].set_xscale('log')
        ax[0].set_xlabel('Log($\\lambda$)')
        ax[0].set_ylabel('Coefficients')
        ax[0].set_title('Coordinate descent')
        plt.axis('tight')

        for i in range(10):
            ax[1].plot(lamda, theta_lasso_gd[i])#label = K[:,i])

        ax[1].set_xscale('log')
        ax[1].set_xlabel('Log($\\lambda$)')
        ax[1].set_title('Gradient descent')
        plt.axis('tight')

        for i in range(10):
            ax[2].plot(lamda, theta_lasso_cg[i])#label = K[:,i])

        ax[2].set_xscale('log')
        ax[2].set_xlabel('Log($\\lambda$)')
        ax[2].set_title('Conjugate gradient descent')
        plt.axis('tight')

In [ ]:
#Plot results
n,_ = theta_lasso_cd.shape
fig,ax = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(10,5))

for i in range(10):
    ax[0].plot(lamda, theta_lasso_cd[i])#,label = K[:,i])

ax[0].set_xscale('log')
ax[0].set_xlabel('Log($\\lambda$)')
ax[0].set_ylabel('Coefficients')
ax[0].set_title('Coordinate descent')
plt.axis('tight')

for i in range(10):
    ax[1].plot(lamda, theta_lasso_gd[i])#label = K[:,i])

ax[1].set_xscale('log')
ax[1].set_xlabel('Log($\\lambda$)')
ax[1].set_title('Gradient descent')
plt.axis('tight')

for i in range(10):
    ax[2].plot(lamda, theta_lasso_cg[i])#label = K[:,i])

ax[2].set_xscale('log')
ax[2].set_xlabel('Log($\\lambda$)')
ax[2].set_title('Conjugate gradient descent')
plt.axis('tight')